In [28]:
from numpy import *
from os import listdir
import operator
import matplotlib
import matplotlib.pyplot as plt
#创造数据集
def createDataSet():
    group = array([[1.0,1.1],[1.0,1.0],[0,0],[0,0.1]])
    labels = ['A','A','B','B']
    return group,labels
#k近邻算法分类器
def classify0(inX,dataSet,labels,k):
    dataSetSize = dataSet.shape[0]
    diffMat = tile(inX,(dataSetSize,1)) - dataSet
    sqDiffMat = diffMat**2
    sqDistances = sqDiffMat.sum(axis=1)
    distances = sqDistances**0.5
    sortedDistIndicies = distances.argsort()
    classCount={}
    for i in range(k):
        voteLabel = labels[sortedDistIndicies[i]]
        classCount[voteLabel] = classCount.get(voteLabel,0) + 1
    sortedClassCount = sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
    return sortedClassCount[0][0]

In [2]:
group,labels = createDataSet()
classify0([0.8,1.0],group,labels,3)

'A'

# 约会网站
- 读取数据并处理

In [3]:
#读取文件，格式化为分类器可输入格式
def file2matrix(filename):
    fr = open(filename)
    arrayOLines = fr.readlines()
    numberOfLines = len(arrayOLines)
    returnMat = zeros((numberOfLines,3))
    classLabelVector = []
    index = 0
    for line in arrayOLines:
        line = line.strip()
        listFromLine = line.split('\t')
        returnMat[index,:] = listFromLine[0:3]
        classLabelVector.append(int(listFromLine[-1]))
        index += 1
    return returnMat,classLabelVector

In [4]:
#datingDataMat,datingLabels = file2matrix('datingTestSet2.txt')

In [5]:
#datingDataMat

In [6]:
#datingLabels[0:20]

In [7]:
#fig = plt.figure()
#ax =fig.add_subplot(111)
#散点图使用datingDataMat矩阵的第一、第二列数据（个性化颜色区分类别）
#ax.scatter(datingDataMat[:,0],datingDataMat[:,1],15.0*array(datingLabels),15.0*array(datingLabels))
#plt.show()

In [8]:
#归一化特征值
def autoNorm(dataSet):
    minVals = dataSet.min(0)
    maxVals = dataSet.max(0)
    ranges = maxVals - minVals
    normDataSet = zeros(shape(dataSet))
    m = dataSet.shape[0]
    normDataSet = dataSet - tile(minVals,(m,1))
    normDataSet = normDataSet/tile(ranges,(m,1))
    return normDataSet,ranges,minVals

In [9]:
#normMat,ranges,minVals = autoNorm(datingDataMat)

In [10]:
#normMat

In [11]:
#ranges

In [12]:
#minVals

- 测试算法

In [13]:
def datingClassTest():
    hoRatio = 0.10
    datingDataMat,datingLabels = file2matrix('datingTestSet2.txt')
    normMat,ranges,minVals = autoNorm(datingDataMat)
    m = normMat.shape[0]
    numTestVecs = int(m*hoRatio)
    errorCount = 0.0
    for i in range(numTestVecs):
        classifierResult = classify0(normMat[i,:],normMat[numTestVecs:m,:],datingLabels[numTestVecs:m],3)
        print ("the classifier came back with %d,the real answer is: %d" % (classifierResult,datingLabels[i]))
        if(classifierResult != datingLabels[i]):errorCount += 1.0
    print ("the total error rate is:%f" % (errorCount/float(numTestVecs)))

In [14]:
datingClassTest()

the classifier came back with 3,the real answer is: 3
the classifier came back with 2,the real answer is: 2
the classifier came back with 1,the real answer is: 1
the classifier came back with 1,the real answer is: 1
the classifier came back with 1,the real answer is: 1
the classifier came back with 1,the real answer is: 1
the classifier came back with 3,the real answer is: 3
the classifier came back with 3,the real answer is: 3
the classifier came back with 1,the real answer is: 1
the classifier came back with 3,the real answer is: 3
the classifier came back with 1,the real answer is: 1
the classifier came back with 1,the real answer is: 1
the classifier came back with 2,the real answer is: 2
the classifier came back with 1,the real answer is: 1
the classifier came back with 1,the real answer is: 1
the classifier came back with 1,the real answer is: 1
the classifier came back with 1,the real answer is: 1
the classifier came back with 1,the real answer is: 1
the classifier came back wit

- 使用算法

In [15]:
def classifyPerson():
    resultList = ['not at all','in small doses','in large doses']
    percentTats = float(input("percentage of time spent playing video games?"))
    ffMiles = float(input("frequent flier miles earned per year?"))
    iceCream = float(input("liters of ice cream consumed per year?"))
    datingDataMat,datingLabels = file2matrix('datingTestSet2.txt')
    normMat,ranges,minVals = autoNorm(datingDataMat)
    inArr = array([ffMiles,percentTats,iceCream])
    classifierResult = classify0((inArr-minVals)/ranges,normMat,datingLabels,3)
    print ("You will probably like this person: ",resultList[classifierResult - 1])

In [16]:
classifyPerson()

percentage of time spent playing video games?10
frequent flier miles earned per year?10
liters of ice cream consumed per year?10
You will probably like this person:  in small doses


# 手写识别系统
- 图像转化为向量

In [17]:
def img2vector(filename):
    returnVect = zeros((1,1024))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0,i*32+j] = int(lineStr[j])
    return returnVect

In [21]:
testVector = img2vector('trainingDigits/0_44.txt')

In [29]:
type(testVector)

numpy.ndarray

In [30]:
testVector.shape

(1, 1024)

In [33]:
len(testVector[0, 0:2])

2

In [34]:
len(testVector[0,32:63])

31

In [48]:
def handwritingCLassTest():
    hwLabels = []
    trainingFileList = listdir('trainingDigits/')
    m = len(trainingFileList)
    trainingMat = zeros((m,1024))
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        hwLabels.append(classNumStr)
        trainingMat[i,:] = img2vector('trainingDigits/%s' % fileNameStr)
    testFileList = listdir('testDigits/')
    errorCount = 0.0
    mTest = len(testFileList)
    for i in range(mTest):
        fileNameStr = testFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        vectorUnderTest = img2vector('testDigits/%s' % fileNameStr)
        classifierResult = classify0(vectorUnderTest,trainingMat,hwLabels,10)
        print ("the classifier came back with:%d,the real answer is:%d" % (classifierResult,classNumStr))
        if(classifierResult != classNumStr): errorCount += 1.0
    print ("\nthe total number of errors is:%d" % errorCount)
    print ("\nthe total error rate is:%f" % (errorCount/float(mTest)))

In [49]:
handwritingCLassTest()

the classifier came back with:7,the real answer is:7
the classifier came back with:4,the real answer is:4
the classifier came back with:1,the real answer is:1
the classifier came back with:4,the real answer is:4
the classifier came back with:7,the real answer is:7
the classifier came back with:3,the real answer is:3
the classifier came back with:3,the real answer is:3
the classifier came back with:6,the real answer is:6
the classifier came back with:4,the real answer is:4
the classifier came back with:9,the real answer is:9
the classifier came back with:8,the real answer is:8
the classifier came back with:5,the real answer is:5
the classifier came back with:0,the real answer is:0
the classifier came back with:4,the real answer is:4
the classifier came back with:9,the real answer is:9
the classifier came back with:2,the real answer is:2
the classifier came back with:4,the real answer is:4
the classifier came back with:0,the real answer is:0
the classifier came back with:2,the real answe

the classifier came back with:7,the real answer is:7
the classifier came back with:2,the real answer is:2
the classifier came back with:1,the real answer is:1
the classifier came back with:5,the real answer is:5
the classifier came back with:2,the real answer is:2
the classifier came back with:2,the real answer is:2
the classifier came back with:7,the real answer is:7
the classifier came back with:5,the real answer is:5
the classifier came back with:1,the real answer is:1
the classifier came back with:3,the real answer is:3
the classifier came back with:2,the real answer is:2
the classifier came back with:0,the real answer is:0
the classifier came back with:7,the real answer is:7
the classifier came back with:4,the real answer is:4
the classifier came back with:2,the real answer is:2
the classifier came back with:3,the real answer is:3
the classifier came back with:4,the real answer is:4
the classifier came back with:6,the real answer is:6
the classifier came back with:8,the real answe

the classifier came back with:5,the real answer is:5
the classifier came back with:4,the real answer is:4
the classifier came back with:1,the real answer is:8
the classifier came back with:4,the real answer is:4
the classifier came back with:4,the real answer is:4
the classifier came back with:3,the real answer is:3
the classifier came back with:5,the real answer is:5
the classifier came back with:7,the real answer is:7
the classifier came back with:9,the real answer is:9
the classifier came back with:3,the real answer is:3
the classifier came back with:5,the real answer is:5
the classifier came back with:8,the real answer is:8
the classifier came back with:8,the real answer is:8
the classifier came back with:2,the real answer is:2
the classifier came back with:1,the real answer is:1
the classifier came back with:9,the real answer is:9
the classifier came back with:5,the real answer is:5
the classifier came back with:2,the real answer is:2
the classifier came back with:9,the real answe

the classifier came back with:2,the real answer is:2
the classifier came back with:5,the real answer is:5
the classifier came back with:1,the real answer is:1
the classifier came back with:2,the real answer is:2
the classifier came back with:5,the real answer is:5
the classifier came back with:4,the real answer is:4
the classifier came back with:9,the real answer is:9
the classifier came back with:0,the real answer is:0
the classifier came back with:5,the real answer is:5
the classifier came back with:9,the real answer is:9
the classifier came back with:7,the real answer is:7
the classifier came back with:0,the real answer is:0
the classifier came back with:0,the real answer is:0
the classifier came back with:1,the real answer is:1
the classifier came back with:8,the real answer is:8
the classifier came back with:4,the real answer is:4
the classifier came back with:0,the real answer is:0
the classifier came back with:6,the real answer is:6
the classifier came back with:7,the real answe

the classifier came back with:6,the real answer is:6
the classifier came back with:8,the real answer is:8
the classifier came back with:6,the real answer is:6
the classifier came back with:8,the real answer is:8
the classifier came back with:8,the real answer is:8
the classifier came back with:0,the real answer is:4
the classifier came back with:6,the real answer is:5
the classifier came back with:6,the real answer is:6
the classifier came back with:8,the real answer is:8
the classifier came back with:7,the real answer is:7
the classifier came back with:6,the real answer is:6
the classifier came back with:4,the real answer is:4
the classifier came back with:3,the real answer is:3
the classifier came back with:7,the real answer is:7
the classifier came back with:2,the real answer is:2
the classifier came back with:0,the real answer is:0
the classifier came back with:7,the real answer is:7
the classifier came back with:4,the real answer is:4
the classifier came back with:6,the real answe

the classifier came back with:4,the real answer is:4
the classifier came back with:9,the real answer is:9
the classifier came back with:9,the real answer is:9
the classifier came back with:7,the real answer is:7
the classifier came back with:9,the real answer is:9
the classifier came back with:0,the real answer is:0
the classifier came back with:6,the real answer is:6
the classifier came back with:8,the real answer is:8
the classifier came back with:5,the real answer is:5
the classifier came back with:8,the real answer is:8
the classifier came back with:0,the real answer is:0
the classifier came back with:1,the real answer is:8
the classifier came back with:1,the real answer is:1
the classifier came back with:2,the real answer is:2
the classifier came back with:7,the real answer is:7
the classifier came back with:1,the real answer is:1
the classifier came back with:1,the real answer is:1
the classifier came back with:0,the real answer is:0
the classifier came back with:2,the real answe

the classifier came back with:1,the real answer is:1
the classifier came back with:5,the real answer is:5
the classifier came back with:8,the real answer is:8
the classifier came back with:1,the real answer is:1
the classifier came back with:9,the real answer is:9
the classifier came back with:7,the real answer is:7
the classifier came back with:3,the real answer is:3

the total number of errors is:20

the total error rate is:0.021142
